# Predict Emotion Valence Score (Fitbit Data + Survey Data, Split by Subject)

In [0]:
import pandas as pd
import numpy as np
!pip install catboost
import catboost
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GroupKFold, cross_val_score, cross_val_predict

# Linear Regression
# import statsmodels.api as sm

# Logistic Regression & Linear Regression
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, BayesianRidge, SGDRegressor
from sklearn import preprocessing
# Random Forest
from sklearn.ensemble import RandomForestClassifier

# Evaluation
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score, auc, roc_curve
import seaborn as sns

# Re-sample
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE

In [0]:
# pd.set_option('display.max_rows', 1000)

## 1.Load Data

In [0]:
exp = pd.read_csv('exp_steps_hr_sleep_survey.csv')

## 2.EDA

### (1) Missing Data Detection

In [0]:
missing_rate = exp.isna().sum()/exp.shape[0]
missing_rate

In [0]:
# Columns with more than 50% missing
missing_mt_half_col = missing_rate[missing_rate > .5].index.values
missing_mt_half_col

In [0]:
# Columns with less than 50% missing
missing_lt_half_col = missing_rate[missing_rate <= .3].index.values
missing_lt_half_col

In [0]:
# Missingness of label
exp[['la_p', 'ha_p', 'ha_n', 'la_n','la', 'p', 'n', 'ha']].isna().sum()

### (2) Observe Labels

In [0]:
exp[['la_p', 'ha_p', 'ha_n', 'la_n','la', 'p', 'n', 'ha']].hist(figsize=(20,12))

In [0]:
corr = exp[['la_p', 'ha_p', 'ha_n', 'la_n','la', 'p', 'n', 'ha']].corr()

In [0]:
def correlation_heatmap(df):
  # Generate a mask for the upper triangle
  mask = np.triu(np.ones_like(df, dtype=np.bool))
  # Set up the matplotlib figure
  f, ax = plt.subplots(figsize=(11, 9))
  # Generate a custom diverging colormap
  cmap = sns.diverging_palette(220, 10, as_cmap=True)
  # Draw the heatmap with the mask and correct aspect ratio
  sns.heatmap(df, mask=mask, cmap=cmap, vmax=.3, center=0,
              square=True, linewidths=.5, cbar_kws={"shrink": .5})

correlation_heatmap(corr)

In [0]:
exp[['la_p', 'ha_p', 'ha_n', 'la_n','la', 'p', 'n', 'ha']].describe()

### (3) Observe Positive / Negative Emotion

In [0]:
# 'pn_raw' represent how happy / unhappy the subjects are
exp['pn_raw'] = (exp['la_p'] + exp['p'] + exp['ha_p'])/3 - (exp['la_n'] + exp['n'] + exp['ha_n'])/3

In [0]:
# Ratio of 'pn_raw' < 0
print('pn_raw < 0: ',str(sum(exp['pn_raw'] < 0)/exp.shape[0]))
print('pn_raw = 0: ',str(sum(exp['pn_raw'] == 0)/exp.shape[0]))
print('pn_raw > 0: ',str(sum(exp['pn_raw'] > 0)/exp.shape[0]))

In [0]:
# Histogram of 'pn_raw'
exp['pn_raw'].hist()

## 3.Pre-processing

### (1) Data Cleaning
Let's only use predictors with less than 50% missing.

In [0]:
# Columns with less than 50% missing
feature_cols = [
       'step_max', 'step_min', 'step_median', 
       
       'steps_max_3h', 'steps_min_3h', 'steps_mean_3h', 'steps_var_3h', 'steps_median_3h', 
       'move_rate_3h', 'active_rate_3h', 'very_active_rate_3h', 'running_rate_3h',
       
       'steps_max_1h', 'steps_min_1h', 'steps_mean_1h', 'steps_var_1h', 'steps_median_1h', 
       'move_rate_1h', 'active_rate_1h', 'very_active_rate_1h', 'running_rate_1h', 
       
       'steps_max_30m', 'steps_min_30m', 'steps_mean_30m', 'steps_var_30m', 'steps_median_30m', 
       'move_rate_30m', 'active_rate_30m', 'very_active_rate_30m', 'running_rate_30m', 
       
       'steps_max_10m', 'steps_min_10m', 'steps_mean_10m', 'steps_var_10m', 'steps_median_10m', 
       'move_rate_10m', 'active_rate_10m', 'very_active_rate_10m', 'running_rate_10m', 
       
       'steps_max_5m', 'steps_min_5m', 'steps_mean_5m', 'steps_var_5m', 'steps_median_5m',
       'move_rate_5m', 'active_rate_5m', 'very_active_rate_5m', 'running_rate_5m', 
       
       'hr_max', 'hr_min', 'hr_med',
       
       'SDNN_3h', 'pHR2_3h', 'rMSSD_3h', 'low_hr_3h', 'high_hr_3h', 'l_h_3h', 'CR_3h', 'hr_mean_3h', 
       'hr_var_3h', 'hr_std_3h', 'hr_median_3h', 'hr_rest_rate_3h', 'hr_moderate_rate_3h',
       'hr_very_active_rate_3h', 
       
       'SDNN_1h', 'pHR2_1h', 'rMSSD_1h', 'low_hr_1h', 'high_hr_1h', 'l_h_1h', 'CR_1h', 'hr_mean_1h',
       'hr_var_1h', 'hr_std_1h', 'hr_median_1h', 'hr_rest_rate_1h',
       'hr_moderate_rate_1h', 'hr_very_active_rate_1h', 
       
       'SDNN_30m', 'pHR2_30m', 'rMSSD_30m', 'low_hr_30m', 'high_hr_30m', 'l_h_30m', 'CR_30m', 'hr_mean_30m', 
       'hr_var_30m', 'hr_std_30m', 'hr_median_30m', 'hr_rest_rate_30m', 
       'hr_moderate_rate_30m', 'hr_very_active_rate_30m', 
       
       'SDNN_10m', 'pHR2_10m', 'rMSSD_10m', 'low_hr_10m', 'high_hr_10m', 'l_h_10m', 'CR_10m', 'hr_mean_10m',
       'hr_var_10m', 'hr_std_10m', 'hr_median_10m', 'hr_rest_rate_10m',
       'hr_moderate_rate_10m', 'hr_very_active_rate_10m', 
       
       'SDNN_5m', 'pHR2_5m', 'rMSSD_5m', 'low_hr_5m', 'high_hr_5m', 'l_h_5m',
       'CR_5m', 'hr_mean_5m', 'hr_var_5m', 'hr_std_5m', 'hr_median_5m',
       'hr_rest_rate_5m', 'hr_moderate_rate_5m', 'hr_very_active_rate_5m',
       
       'hr_0', 'hr_0.3', 'hr_0.5', 'hr_0.8', 'hr_1', 
       
       'SBQ', 'FTP', 'SWLS', 'Neuroticism', 'Extraversion', 'Conscientiousness',
       'NS_total', 'BIS_total', 'BIS.5', 'BAS_D', 'BAS_FS', 'BAS_RR',
       'HAP_actual', 'P_actual', 'LAP_actual', 'LA_actual', 'LAN_actual',
       'N_actual', 'HAN_actual', 'HA_actual', 'HAP_ideal', 'P_ideal',
       'LAP_ideal', 'LA_ideal', 'LAN_ideal', 'N_ideal', 'HAN_ideal',
       'HA_ideal', 'Education', 'Ethnicity', 'Sex', 'Marital_Status',
       'Children', 'Household_income', 'Age', 'BMI', 'Medications', 
       'survey_hour','experiment', 'subject'
       ]
label_cols = ['la_p', 'ha_p', 'ha_n', 'la_n', 'la', 'p', 'n', 'ha']

In [0]:
# Generate design matrix
design_matrix = exp[feature_cols + label_cols]

In [0]:
# Drop NA
design_matrix = design_matrix.dropna()

In [0]:
design_matrix.shape

### (2) Generate Labels

In [0]:
design_matrix['pn_raw'] = (design_matrix['la_p'] + design_matrix['p'] + design_matrix['ha_p'])/3 \
        - (design_matrix['la_n'] + design_matrix['n'] + design_matrix['ha_n'])/3

In [0]:
# Create dummy variable for valence, with 0 to be unhappy, and 1 to be happy.
design_matrix['valence'] = 1
design_matrix.loc[design_matrix['pn_raw'] <= 0, 'valence'] = 0

In [0]:
design_matrix['valence'].hist()

As you can see, this is very imbalanced data. Let's use a logistic regression to start.

### (3) Encode Categorical Data

i. List All Categorical Data

In [0]:
# Categorical Features
design_matrix.columns[design_matrix.dtypes != 'float64']

ii. Convert Education

In [0]:
design_matrix['Education'].unique()

MBA = ['MBA', 'High School Diploma, BA, MBA', 'B.S. Business Management', 'BS Business','College grad']
# Art = ['Bachelor of Arts', 'Bachelor of Arts + Equivalency Degree', 'Bachelor Degree Music Ed']
# Med = ['Medical Degree', 'B.A. English/Spanish B.S. Nursing', 'BSN- bachelors of science in nursing']
# Psy = ['Education specialist in School Psy, M.A. in Educational Psy', 'M.S. Psychology/ B.A. Sociology', 'B.S. Psychology Texas A&M']


HighSchool = ['12.0','high school', 'High School', 'high school + cosmetology school', 'High School Graduate']
BA = ['Some college', 'B.A. English/Spanish B.S. Nursing', 'B.S. working on MD and MTS', 'B.S.  Mechanical Engineering', 
      "Bachelor's Degree", 'B.A.', 'College student 3 years', 'B.A. from Cornell Univ.', 'Bachelors degree in Social Work', 
      'BS', 'BA Vanderbilt', 'BS Communications', "college graduate (bachelor's degree in Music from Belmont University)",
      'Bachelor of Arts', 'BSN- bachelors of science in nursing', 'BSE Princeton',
      'Obtained B.S., 2011', 'bachelor of science', 'Bachelor of Social Work', 'B.S.', 'BBA - Bachelors', 'BSC Communications',
      'B.A. 1988', 'B.S. degree dietetics, internship', 'Bachelor of Arts + Equivalency Degree', 'Bachelors', 'B.S. Business Management',
      'B.S. Psychology Texas A&M', 'BS Journalism; BS Nutrition', 'B.E.Sc.', 'Bachelor Degree Music Ed', 'NDCDP',
      'BS Math/Computer Science', 'BS Business', 'college graduate','14.0','16.0']
Master = MBA + ['College, Masters',"Bachelor of science, 1 year of Master's",'Masters; PhD ongoing', 'M.S. Psychology/ B.A. Sociology',
                'B.S., MPH','18.0', '19.0']
PhD = ['Ph.D.', 'PhD Candidate', 'PhD', 'BS. (PhD Student currently)', 'Masters; PhD ongoing', 
       'Pursuing PhD at Vanderbilt. Have B.S. from UTK', 'BA from Stanford, JD from Univ of Michigan','21.0', '22.0', '20.0']

In [0]:
# design_matrix['is_MBA'] = 0
# design_matrix.loc[design_matrix['Education'].isin(MBA), 'is_MBA'] = 1
# design_matrix['is_Art'] = 0
# design_matrix.loc[design_matrix['Education'].isin(Art), 'is_Art'] = 1
# design_matrix['is_Med'] = 0
# design_matrix.loc[design_matrix['Education'].isin(Med), 'is_Med'] = 1
# design_matrix['is_Psy'] = 0
# design_matrix.loc[design_matrix['Education'].isin(Psy), 'is_Psy'] = 1

design_matrix['edu_hs'] = 0
design_matrix.loc[design_matrix['Education'].isin(HighSchool), 'edu_hs'] = 1
design_matrix['edu_ba'] = 0
design_matrix.loc[design_matrix['Education'].isin(BA), 'edu_ba'] = 1
design_matrix['edu_ma'] = 0
design_matrix.loc[design_matrix['Education'].isin(Master), 'edu_ma'] = 1
design_matrix['edu_phd'] = 0
design_matrix.loc[design_matrix['Education'].isin(PhD), 'edu_phd'] = 1

iii. Convert Ethnicity

In [0]:
design_matrix['Ethnicity'].unique()

In [0]:
design_matrix['is_wht'] = 0
design_matrix.loc[design_matrix['Ethnicity'] == 'White', 'is_wht'] = 1
design_matrix['is_hsp_other'] = 0
design_matrix.loc[design_matrix['Ethnicity'] == 'Hispanic/Other', 'is_hsp_other'] = 1
design_matrix['is_blk'] = 0
design_matrix.loc[design_matrix['Ethnicity'] == 'Black', 'is_blk'] = 1
design_matrix['is_hsp_caucasian'] = 0
design_matrix.loc[design_matrix['Ethnicity'] == 'Hispanic/Caucasian', 'is_hsp_caucasian'] = 1
design_matrix['is_birace'] = 0
design_matrix.loc[design_matrix['Ethnicity'].isin(['Biracial', 'More than 1 race']), 'is_birace'] = 1
design_matrix['is_hsp'] = 0
design_matrix.loc[design_matrix['Ethnicity'] == 'Hispanic', 'is_hsp'] = 1
design_matrix['is_asn'] = 0
design_matrix.loc[design_matrix['Ethnicity'] == 'Asian', 'is_asn'] = 1
design_matrix['is_white_pacific'] = 0
design_matrix.loc[design_matrix['Ethnicity'] == 'White/Pacific Islander', 'is_white_pacific'] = 1
design_matrix['is_hwi'] = 0
design_matrix.loc[design_matrix['Ethnicity'] == 'Native Hawaiian/Pacific Islander', 'is_hwi'] = 1

iv. Sex

In [0]:
design_matrix['Sex'].unique()

In [0]:
design_matrix['is_female'] = 0
design_matrix.loc[design_matrix['Sex'] == 'Female', 'is_female'] = 1

v. Marital_Status

In [0]:
design_matrix['Marital_Status'].unique()

In [0]:
design_matrix['is_married'] = 0
design_matrix.loc[design_matrix['Marital_Status'] == 'Married', 'is_married'] = 1
design_matrix['is_divorced'] = 0
design_matrix.loc[design_matrix['Marital_Status'] == 'Divorced', 'is_divorced'] = 1
design_matrix['is_single'] = 0
design_matrix.loc[design_matrix['Marital_Status'] == 'Single', 'is_single'] = 1
design_matrix['is_widowed'] = 0
design_matrix.loc[design_matrix['Marital_Status'] == 'Widowed', 'is_widowed'] = 1
design_matrix['is_with_partner'] = 0
design_matrix.loc[design_matrix['Marital_Status'] == 'Living with partner', 'is_with_partner'] = 1

vi. Household_income

In [0]:
design_matrix['Household_income'].unique()

In [0]:
design_matrix['income'] = 0
design_matrix.loc[design_matrix['Household_income'] == '$10,000-$19,999', 'income'] = 1
design_matrix.loc[design_matrix['Household_income'] == '$20,000-$29,999', 'income'] = 2
design_matrix.loc[design_matrix['Household_income'] == '$30,000-$39,999', 'income'] = 3
design_matrix.loc[design_matrix['Household_income'] == '$40,000-$49,999', 'income'] = 4
design_matrix.loc[design_matrix['Household_income'] == '$50,000-$59,999', 'income'] = 5
design_matrix.loc[design_matrix['Household_income'] == '$60,000-$69,999', 'income'] = 6
design_matrix.loc[design_matrix['Household_income'] == '$70,000-$79,999', 'income'] = 7
design_matrix.loc[design_matrix['Household_income'] == '$80,000-$89,999', 'income'] = 8
design_matrix.loc[design_matrix['Household_income'] == '$90,000-$99,999', 'income'] = 9
design_matrix.loc[design_matrix['Household_income'] == '$100,000-$109,999', 'income'] = 10
design_matrix.loc[design_matrix['Household_income'] == '$110,000-$119,999', 'income'] = 11
design_matrix.loc[design_matrix['Household_income'] == '$120,000-$129,999', 'income'] = 12
design_matrix.loc[design_matrix['Household_income'] == '$130,000-$139,000', 'income'] = 13
design_matrix.loc[design_matrix['Household_income'] == '$140,000-$149,999', 'income'] = 14
design_matrix.loc[design_matrix['Household_income'].isin(['$150,000 or more ','$150,000 or more']), 'income'] = 15

vii. Medications

In [0]:
design_matrix['Medications'].unique()

In [0]:
design_matrix['have_med'] = 1
design_matrix.loc[design_matrix['Medications'] == 'None', 'have_med'] = 0

viii. Exp

In [0]:
# 'DND' = 0, 'ROO' = 1
design_matrix['exp'] = 1
design_matrix.loc[design_matrix['experiment'] == 'DND', 'exp'] = 0

In [0]:
a = [1,2,3]
a.remove(2)
a

In [0]:
# Update Feature Cols

features_common = ['step_max', 'step_min', 'step_median', 
       
       'steps_max_3h', 'steps_min_3h', 'steps_mean_3h', 'steps_var_3h', 'steps_median_3h', 
       'move_rate_3h', 'active_rate_3h', 'very_active_rate_3h', 'running_rate_3h',
       
       'steps_max_1h', 'steps_min_1h', 'steps_mean_1h', 'steps_var_1h', 'steps_median_1h', 
       'move_rate_1h', 'active_rate_1h', 'very_active_rate_1h', 'running_rate_1h', 
       
       'steps_max_30m', 'steps_min_30m', 'steps_mean_30m', 'steps_var_30m', 'steps_median_30m', 
       'move_rate_30m', 'active_rate_30m', 'very_active_rate_30m', 'running_rate_30m', 
       
       'steps_max_10m', 'steps_min_10m', 'steps_mean_10m', 'steps_var_10m', 'steps_median_10m', 
       'move_rate_10m', 'active_rate_10m', 'very_active_rate_10m', 'running_rate_10m', 
       
       'steps_max_5m', 'steps_min_5m', 'steps_mean_5m', 'steps_var_5m', 'steps_median_5m',
       'move_rate_5m', 'active_rate_5m', 'very_active_rate_5m', 'running_rate_5m', 
       
       'hr_max', 'hr_min', 'hr_med',
       
       'SDNN_3h', 'pHR2_3h', 'rMSSD_3h', 'low_hr_3h', 'high_hr_3h', 'l_h_3h', 'CR_3h', 'hr_mean_3h', 
       'hr_var_3h', 'hr_std_3h', 'hr_median_3h', 'hr_rest_rate_3h', 'hr_moderate_rate_3h',
       'hr_very_active_rate_3h', 
       
       'SDNN_1h', 'pHR2_1h', 'rMSSD_1h', 'low_hr_1h', 'high_hr_1h', 'l_h_1h', 'CR_1h', 'hr_mean_1h',
       'hr_var_1h', 'hr_std_1h', 'hr_median_1h', 'hr_rest_rate_1h',
       'hr_moderate_rate_1h', 'hr_very_active_rate_1h', 
       
       'SDNN_30m', 'pHR2_30m', 'rMSSD_30m', 'low_hr_30m', 'high_hr_30m', 'l_h_30m', 'CR_30m', 'hr_mean_30m', 
       'hr_var_30m', 'hr_std_30m', 'hr_median_30m', 'hr_rest_rate_30m', 
       'hr_moderate_rate_30m', 'hr_very_active_rate_30m', 
       
       'SDNN_10m', 'pHR2_10m', 'rMSSD_10m', 'low_hr_10m', 'high_hr_10m', 'l_h_10m', 'CR_10m', 'hr_mean_10m',
       'hr_var_10m', 'hr_std_10m', 'hr_median_10m', 'hr_rest_rate_10m',
       'hr_moderate_rate_10m', 'hr_very_active_rate_10m', 
       
       'SDNN_5m', 'pHR2_5m', 'rMSSD_5m', 'low_hr_5m', 'high_hr_5m', 'l_h_5m',
       'CR_5m', 'hr_mean_5m', 'hr_var_5m', 'hr_std_5m', 'hr_median_5m',
       'hr_rest_rate_5m', 'hr_moderate_rate_5m', 'hr_very_active_rate_5m',
       
       'hr_0', 'hr_0.3', 'hr_0.5', 'hr_0.8', 'hr_1', 
       
       'SBQ', 'FTP', 'SWLS', 'Neuroticism', 'Extraversion', 'Conscientiousness',
       'NS_total', 'BIS_total', 'BIS.5', 'BAS_D', 'BAS_FS', 'BAS_RR',
       'HAP_actual', 'P_actual', 'LAP_actual', 'LA_actual', 'LAN_actual',
       'N_actual', 'HAN_actual', 'HA_actual', 'HAP_ideal', 'P_ideal',
       'LAP_ideal', 'LA_ideal', 'LAN_ideal', 'N_ideal', 'HAN_ideal',
       'HA_ideal', 'survey_hour'
       
       ]

features_lite = ['step_max', 'step_min', 'step_median', 
       
      #  'steps_max_3h', 'steps_min_3h', 'steps_mean_3h', 'steps_var_3h', 'steps_median_3h', 
      #  'move_rate_3h', 'active_rate_3h', 'very_active_rate_3h', 'running_rate_3h',
       
       'steps_max_1h', 'steps_min_1h', 
       'steps_var_1h', 'steps_median_1h', 
       'active_rate_1h', 'running_rate_1h',  
       #'move_rate_1h', 'very_active_rate_1h', 'steps_mean_1h', 
       
      #  'steps_max_30m', 'steps_min_30m', 'steps_mean_30m', 'steps_var_30m', 'steps_median_30m', 
      #  'move_rate_30m', 'active_rate_30m', 'very_active_rate_30m', 'running_rate_30m', 
       
       'steps_max_10m', 'steps_min_10m', 
       'steps_var_10m', 'steps_median_10m', 
       'move_rate_10m', 
       # 'very_active_rate_10m', 'active_rate_10m', 'steps_mean_10m',
       'running_rate_10m', 
       
      #  'steps_max_5m', 'steps_min_5m', 'steps_mean_5m', 'steps_var_5m', 'steps_median_5m',
      #  'move_rate_5m', 'active_rate_5m', 'very_active_rate_5m', 'running_rate_5m', 
       
       'hr_max', 'hr_min', 'hr_med',
       
       'SDNN_3h', 'pHR2_3h', 'rMSSD_3h', 'low_hr_3h', 'high_hr_3h', 'l_h_3h', 'CR_3h', 
      #  'hr_mean_3h', 
      #  'hr_var_3h', 'hr_std_3h', 'hr_median_3h', 'hr_rest_rate_3h', 'hr_moderate_rate_3h',
      #  'hr_very_active_rate_3h', 
       
       'SDNN_1h', 'pHR2_1h', 'high_hr_1h', 'l_h_1h', 'hr_mean_1h',
       #'rMSSD_1h', 'low_hr_1h', 'CR_1h', 'hr_median_1h', 'hr_var_1h', 
       'hr_std_1h', 'hr_rest_rate_1h',
       'hr_moderate_rate_1h', 'hr_very_active_rate_1h', 
       
       'SDNN_30m', 'pHR2_30m', 'rMSSD_30m', 'low_hr_30m', 'high_hr_30m', 'l_h_30m', 'CR_30m', 
      #  'hr_mean_30m', 
      #  'hr_var_30m', 'hr_std_30m', 'hr_median_30m', 'hr_rest_rate_30m', 
      #  'hr_moderate_rate_30m', 'hr_very_active_rate_30m', 
       
       'SDNN_10m', 'pHR2_10m', 'rMSSD_10m', 'l_h_10m', 
       # 'hr_mean_10m', 'hr_var_10m', 'hr_median_10m', 'low_hr_10m', 'high_hr_10m', 'CR_10m', 
       'hr_std_10m',  'hr_rest_rate_10m',
       'hr_moderate_rate_10m', 'hr_very_active_rate_10m', 
       
       'SDNN_5m', 'pHR2_5m', 'rMSSD_5m', 'low_hr_5m', 'high_hr_5m', 'l_h_5m',
       'CR_5m', 
      #  'hr_mean_5m', 'hr_var_5m', 'hr_std_5m', 'hr_median_5m',
      #  'hr_rest_rate_5m', 'hr_moderate_rate_5m', 'hr_very_active_rate_5m',
       
       'hr_0', 'hr_0.3', 'hr_0.8', 'hr_1', # 'hr_0.5',
       
       'SBQ', 'SWLS', 'Neuroticism', 'Extraversion', 
       'NS_total', 'BIS_total', 'BIS.5', 'BAS_D', 'BAS_FS', 'BAS_RR',
      # 'FTP', 'Conscientiousness',
      #  'HAP_actual', 'P_actual', 'LAP_actual', 'LA_actual', 'LAN_actual',
      #  'N_actual', 'HAN_actual', 'HA_actual', 'HAP_ideal', 'P_ideal',
      #  'LAP_ideal', 'LA_ideal', 'LAN_ideal', 'N_ideal', 'HAN_ideal',
      #  'HA_ideal', 
      'survey_hour'
       
       ]


feature_encode = ['exp', 'income', 'have_med', 'is_female',
                'is_married', 'is_divorced', 'is_single', 'is_widowed', 
                'is_with_partner', 'is_wht', 'is_hsp_other', 'is_blk', 
                'is_hsp_caucasian', 'is_birace', 'is_hsp', 'is_asn', 
                'is_white_pacific', 'is_hwi', 'edu_hs', 'edu_ba', 'edu_ma', 
                'edu_phd']
feature_ca = ['Education', 'Ethnicity', 'Sex', 'Marital_Status','Children', 
              'Household_income', 'Age', 'BMI', 'Medications', 'survey_hour',
              'experiment' ]
feature_num = features_common + feature_encode
feature_num_lite = features_lite + feature_encode
feature_cols_ca = features_common + feature_ca

### (4) Detect Correlation

In [0]:
corr_matrix = design_matrix[feature_cols_update].corr()

In [0]:
corr_matrix_df = pd.DataFrame(corr_matrix.unstack())
corr_matrix_df.columns = ['corr']

In [0]:
# Filter pairs which are highly correlated
corr_matrix_df[(corr_matrix_df['corr'] > 0.9) & (corr_matrix_df['corr'] < 1)]

As we can see, the correlation exist in 'mean' and 'median' fetaures, 'var' and 'std' features. Let's get rid of them by changing feature columns.

In [0]:
feature_cols = ['hr_mean_3h', 'hr_std_3h', 'hr_rest_rate_3h', 
                'hr_mean_1h', 'hr_std_1h', 'hr_rest_rate_1h', 
                'hr_std_30m', 'hr_very_active_rate_30m', 
                'hr_mean_10m', 'hr_std_10m', 'hr_rest_rate_10m', 'hr_very_active_rate_10m',
                'steps_mean_3h', 'steps_var_3h', 'active_rate_3h',
                'steps_var_1h', 'very_active_rate_1h', 
                'steps_mean_30m', 'steps_var_30m', 'active_rate_30m', 
                'steps_var_10m', 'move_rate_10m', 'very_active_rate_10m', 
                'SBQ',  'SWLS', 
                'NS_total', 'BIS_total', 'BIS.5', 'BAS_D',
                'BAS_FS', 'BAS_RR', 'HAP_actual', 
                'LA_actual', 'LAN_actual', 'N_actual', 'HAN_actual', 'HA_actual',
                'P_ideal', 'LAN_ideal',
                'HAN_ideal', 'HA_ideal', 'Children', 'Age',
                'BMI',
                'exp', 'income', 'have_med', 'is_female',
                'is_married', 'is_divorced', 'is_single', 'is_widowed', 'is_with_partner',
                'is_wht', 'is_hsp_other', 'is_blk', 'is_hsp_caucasian', 'is_birace', 'is_hsp', 'is_asn', 'is_white_pacific', 'is_hwi',
                'is_MBA', 'is_Art', 'is_Med', 'is_Psy', 'edu_hs', 'edu_ba', 'edu_ma', 'edu_phd',
                'survey_hour'
                # , 'Conscientiousness', 'HAP_ideal', 'LAP_ideal', 'P_actual', 'LAP_actual', 'FTP', 'Neuroticism', 'Extraversion', 'LA_ideal', 'N_ideal', 
                ]

# feature_cols_update

In [0]:
# Check correlation again
corr_matrix = design_matrix[feature_num_lite].corr()
corr_matrix_df = pd.DataFrame(corr_matrix.unstack())
corr_matrix_df.columns = ['corr']
# Filter pairs which are highly correlated
corr_matrix_df[(corr_matrix_df['corr'] > 0.9) & (corr_matrix_df['corr'] < 1)]

### (5) Split the Data by Subject

In [0]:
# Grouped Data-Split, by subject, prevent information leakage
X = design_matrix[feature_num_lite]
X['intercept'] = 1
y = design_matrix['pn_raw']
group = design_matrix['subject'].values
gkf_modeling = list(GroupKFold(n_splits=4).split(X,y,group))

X_train = X.iloc[gkf_modeling[0][0],]
X_test = X.iloc[gkf_modeling[0][1],]
y_train = y.iloc[gkf_modeling[0][0],]
y_test = y.iloc[gkf_modeling[0][1],]

### (6) Split the Data by Stratification

In [0]:
# Grouped Data-Split, by subject, prevent information leakage
X = design_matrix[feature_num_lite]
y = design_matrix['pn_raw']
X_ran_train, X_ran_test, y_ran_train, y_ran_test = train_test_split(X, y, test_size=0.25, random_state=27)

### (7) Scale the Data

In [0]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 4.Cross Validation

### (1) Scoring criteria

All available scoring criteria can be found here: https://scikit-learn.org/stable/modules/model_evaluation.html

In [0]:
scoring = {
    # 'explained_variance': 'explained_variance',
    'r2': 'r2',
    'rmse':'neg_root_mean_squared_error'
}

### (2) Tools for evaluate results

In [0]:
def avg_dict(dict):
  avgDict = {}
  for k,v in dict.items():
      avgDict[k] = sum(v)/ float(len(v))
  return avgDict

### (3) Cross Validation

Ref: https://stackoverflow.com/questions/35876508/evaluate-multiple-scores-on-sklearn-cross-val-score

In [0]:
from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict
from sklearn import metrics, svm
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
# Information about scoring metrics
models = [LinearRegression(), Ridge(), Lasso(), svm.SVR(), BayesianRidge(), SGDRegressor(), 
          KNeighborsRegressor(n_neighbors=10), DecisionTreeRegressor()]
model_names = ['Linear', 'Ridge', 'Lasso', 'svm', 'BayesianRidge', 'SGDRegressor', 'KNN', 'DecisionTreeRegressor']

for model, name in zip(models, model_names):
  print(name)
  scores = cross_validate(model, X, y, cv=6, scoring=scoring)
  print(avg_dict(scores))

## 5.Baseline Model - Linear Regression

### (1) Modeling

In [0]:
linear_model_1 = sm.OLS(y_train, X_train_scaled).fit()

### (2) Prediction

In [0]:
linear_pred = linear_model_1.predict(X_test_scaled)

### (3) Evaluation

i. Model Summary

In [0]:
linear_model_1.summary()

ii. Prediction Evaluation

In [0]:
# RMSE
np.sqrt(np.mean((linear_pred - y_test)**2))

iii. Prediction vs True Label

In [0]:
def plot_pred_truth(y_test, y_pred):
  plt.figure(figsize=(12,6))
  plt.scatter(y_test, y_pred)
  plt.title('Prediction vs Ground Truth')
  plt.ylabel('Prediction')
  plt.xlabel('Valence')

plot_pred_truth(y_test, linear_pred)

iv. Convert the Regression Result to Classification

In [0]:
def reg_to_class(pred):
  '''
  This function converts regression results to classification.

  Parameters:
  pred (array):     regression results to be converted to classification labels

  Returns:
  Converted classification results
  '''
  return [0 if x < 0 else 1 for x in pred]

linear_pred_class = reg_to_class(linear_pred)
y_test_class = reg_to_class(y_test)

In [0]:
def eval_class(y_test, y_pred):
  '''
  This function evaluates the model performance
  in terms of Accuracy, Recall, Precision, Specificity and F1
  
  Parameters:
  y_test (array):     Test label
  y_pred (array):     Model prediction

  Returns:
  Evaluation information printed
  '''
  # Accuracy
  print('Accuracy: \t', str(accuracy_score(y_test, y_pred)))
  # Recall
  print('Recall: \t', str(recall_score(y_test, y_pred)))
  # Precision
  print('Precision: \t', str(precision_score(y_test, y_pred)))
  # Specificity
  tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
  print('Specificity: \t', str(tn / (tn+fp)))
  # F1
  print('F1: \t\t', str(f1_score(y_test, y_pred)))

In [0]:
def plot_roc(y_test,  prob, model_type):
  '''
  This function plots roc curve for classfication algotrithms.
  
  Parameters:
  y_test (array):       Test label
  prob (array):         Model prediction of probability for binary class
  model_type (string):  Model names

  Returns: 
  RoC plot
  '''
  preds = prob[:,1]
  fpr, tpr, threshold = roc_curve(y_test, preds)
  roc_auc = auc(fpr, tpr)
  if roc_auc < 0.5:
    preds = prob[:,0]
    fpr, tpr, threshold = roc_curve(y_test, preds)
    roc_auc = auc(fpr, tpr)
  # Plot
  plt.title(model_type+ ' for Emotion Classification')
  plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
  plt.legend(loc = 'lower right')
  plt.plot([0, 1], [0, 1],'r--')
  plt.xlim([0, 1])
  plt.ylim([0, 1])
  plt.ylabel('True Positive Rate')
  plt.xlabel('False Positive Rate')
  plt.show()

In [0]:
def confision_matrix(y_test, y_pred, model_type):
  '''
  This function plots confision matrix for classfication algotrithms.
  
  Parameters:
  y_test (array):     Test label
  y_pred (array):     Model prediction
  model_type (string):  Model names

  Returns: 
  Confusion Matrix plot
  '''
  cm_2 = confusion_matrix(y_test, y_pred) 
  cm_2_df = pd.DataFrame(cm_2)

  plt.figure(figsize=(5.5,4))
  sns.heatmap(cm_2_df, annot=True)
  accuracy = accuracy_score(y_test, y_pred)
  plt.title(model_type + ' \nAccuracy:{0:.3f}'.format(accuracy))
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()

In [0]:
# plot_roc(y_test, logistic_pred_probs, 'Linear Regression')
confision_matrix(y_test_class, linear_pred_class, 'Logistic Regression')

In [0]:
eval_class(y_test_class, linear_pred_class)

All results are 1, we are suffering from imbalanced data. Let's try Random Forest.

In [0]:
def avg_dict(dict):
  avgDict = {}
  for k,v in dict.items():
      avgDict[k] = sum(v)/ float(len(v))
  return avgDict

## 5.Random Forest - Less vunerable to imbalanced data

### (1) Modeling

In [0]:
randomforest = RandomForestClassifier(n_estimators=100).fit(X_train, y_train)

###(2) Prediction

In [0]:
randomforest_pred = randomforest.predict(X_test)
randomforest_pred_probs = randomforest.predict_proba(X_test)

In [0]:
randomforest_pred

Prediction '1' still dominates. Let's look at the evaluation.

### (3) Evaluation

In [0]:
eval_class(y_test, randomforest_pred)

In [0]:
plot_roc(y_test, randomforest_pred_probs, 'Random Forest')

In [0]:
confision_matrix(y_test, randomforest_pred, 'Random Forest')

Still not perform well. Let's try to model using random split data.

## 6.CatBoost

Let me just try it.

### (1) Modeling

In [0]:
# Model params
params_1 = {
    'loss_function': 'MultiClass',
    'eval_metric': 'Accuracy',
    'verbose': 10,
    'random_seed': 24,
    # 'depth':6, 
    'learning_rate':0.0001,
    'use_best_model': True,
    'l2_leaf_reg': 10,
    'bagging_temperature': 3,
    'od_type': "Iter",
    'od_wait': 100
}

catboostCls = catboost.CatBoostClassifier(**params_1)
catboostCls.fit(X_train, y_train,
             eval_set = (X_test, y_test),
             use_best_model = True,
             plot=True)

### (2) Prediction

In [0]:
cat_pred = catboostCls.predict(X_test)
cat_prob = catboostCls.predict_proba(X_test)

### (3) Evaluation

In [0]:
eval_class(y_test, cat_pred)

In [0]:
plot_roc(y_test, cat_prob, 'CatBoost')

In [0]:
confision_matrix(y_test, cat_pred, 'CatBoost')

CatBoost is only a little bit better. Maybe we should deal with imbalanced data.

## 6.Over Sample Minority Class

### (1) Upsample Data

In [0]:
# concatenate our training data back together
data_to_sample = pd.concat([X_train, y_train], axis=1)

# separate minority and majority classes
unhappy = data_to_sample[data_to_sample['valence']==0]
happy = data_to_sample[data_to_sample['valence']==1]


# upsample minority
unhappy_upsampled = resample(unhappy,
                          replace=True, # sample with replacement
                          n_samples=len(happy), # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([happy, unhappy_upsampled])

# check new class counts
upsampled['valence'].value_counts()

In [0]:
y_upsample_train = upsampled['valence']
X_upsample_train = upsampled.drop('valence', axis=1)

### (2) Logistic Regression

i. Scale the data

In [0]:
scaler = preprocessing.StandardScaler().fit(X_upsample_train)
X_upsample_train_scaled = scaler.transform(X_upsample_train)
X_test_scaled = scaler.transform(X_test)

ii. Modeling

In [0]:
logisticReg2 = LogisticRegression(max_iter=20000)
logisticReg2.fit(X_upsample_train_scaled, y_upsample_train)

iii. Prediction

In [0]:
logistic2_pred = logisticReg2.predict(X_test_scaled)
logistic2_pred_probs = logisticReg2.predict_proba(X_test_scaled)

iv. Evaluation

In [0]:
eval_class(y_test, logistic2_pred)

As we can see, the specificity score improved drmatically, this is a good sign.

In [0]:
plot_roc(y_test, logistic2_pred_probs, 'Linear Regression')

In [0]:
confision_matrix(y_test, logistic2_pred, 'Linear Regression')

### (3) Random Forest

i. Modeling

In [0]:
randomforest2 = RandomForestClassifier(n_estimators=100).fit(X_upsample_train_scaled, y_upsample_train)

ii.Prediction

In [0]:
randomforest2_pred = randomforest2.predict(X_test_scaled)
randomforest2_pred_probs = randomforest2.predict_proba(X_test_scaled)

iii.Evaluation

In [0]:
eval_class(y_test, randomforest2_pred)

In [0]:
plot_roc(y_test, randomforest2_pred_probs, 'Random Forest')

In [0]:
confision_matrix(y_test, randomforest2_pred, 'Random Forest')

The AUC improved a little bit. But specificity is still 0.

### (4) CatBoost

i. Modeling

In [0]:
# Model params
params_2 = {
    'loss_function': 'MultiClass',
    'eval_metric': 'Accuracy',
    'verbose': 10,
    'random_seed': 24,
    # 'depth':6, 
    'learning_rate':0.0001,
    'use_best_model': True,
    'l2_leaf_reg': 10,
    'bagging_temperature': 3,
    'od_type': "Iter",
    'od_wait': 100
}

catboostCls2 = catboost.CatBoostClassifier(**params_2)
catboostCls2.fit(X_upsample_train_scaled, y_upsample_train,
             eval_set = (X_test_scaled, y_test),
             use_best_model = True,
             plot=True)

ii. Prediction

In [0]:
cat2_pred = catboostCls2.predict(X_test_scaled)
cat2_prob = catboostCls2.predict_proba(X_test_scaled)

iii. Evaluation

In [0]:
eval_class(y_test, cat2_pred)

In [0]:
plot_roc(y_test, cat2_prob, 'CatBoost')

In [0]:
confision_matrix(y_test, cat2_pred, 'CatBoost')

## 7.Under Sample Majority Class

### (1) Undersample Data

In [0]:
# downsample majority
happy_downsampled = resample(happy,
                                replace = False, # sample without replacement
                                n_samples = len(unhappy), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled = pd.concat([happy_downsampled, unhappy])

# checking counts
downsampled['valence'].value_counts()

In [0]:
y_downsample_train = downsampled['valence']
X_downsample_train = downsampled.drop('valence', axis=1)

### (2) Logistic Regression

i. Scale the Data

In [0]:
scaler = preprocessing.StandardScaler().fit(X_downsample_train)
X_downsample_train_scaled = scaler.transform(X_downsample_train)
X_test_scaled = scaler.transform(X_test)

ii. Modeling

In [0]:
logisticReg3 = LogisticRegression(max_iter=20000)
logisticReg3.fit(X_downsample_train_scaled, y_downsample_train)

iii. Prediction

In [0]:
logistic3_pred = logisticReg3.predict(X_test_scaled)
logistic3_pred_probs = logisticReg3.predict_proba(X_test_scaled)

iv. Evaluation

In [0]:
eval_class(y_test, logistic3_pred)

In [0]:
plot_roc(y_test, logistic3_pred_probs, 'Linear Regression')

In [0]:
confision_matrix(y_test, logistic3_pred, 'Linear Regression')

### (3) Random Forest

i. Modeling

In [0]:
randomforest3 = RandomForestClassifier(n_estimators=100).fit(X_downsample_train_scaled, y_downsample_train)

ii. Prediction

In [0]:
randomforest3_pred = randomforest3.predict(X_test_scaled)
randomforest3_pred_probs = randomforest3.predict_proba(X_test_scaled)

iii. Evaluation

In [0]:
eval_class(y_test, randomforest3_pred)

In [0]:
plot_roc(y_test, randomforest3_pred_probs, 'Random Forest')

In [0]:
confision_matrix(y_test, randomforest3_pred, 'Random Forest')

### (4) CatBoost

i. Modeling

In [0]:
# Model params
params_3 = {
    'loss_function': 'MultiClass',
    'eval_metric': 'Accuracy',
    'verbose': 10,
    'random_seed': 24,
    # 'depth':6, 
    'learning_rate':0.0001,
    'use_best_model': True,
    'l2_leaf_reg': 10,
    'bagging_temperature': 3,
    'od_type': "Iter",
    'od_wait': 100
}

catboostCls3 = catboost.CatBoostClassifier(**params_3)
catboostCls3.fit(X_downsample_train_scaled, y_downsample_train,
             eval_set = (X_test_scaled, y_test),
             use_best_model = True,
             plot=True)

ii. Prediction

In [0]:
cat3_pred = catboostCls3.predict(X_test_scaled)
cat3_prob = catboostCls3.predict_proba(X_test_scaled)

iii. Evaluation

In [0]:
eval_class(y_test, cat3_pred)

In [0]:
plot_roc(y_test, cat3_prob, 'CatBoost')

In [0]:
confision_matrix(y_test, cat3_pred, 'CatBoost')

## 8.Generate Synthetic Samples (SMOTE)

### (1) Generate New Samples

In [0]:
sm = SMOTE(random_state=27, ratio=1.0)
X_smo_train, y_smo_train = sm.fit_sample(X_train, y_train)

### (2) Logistic Regression

i. Scale the Data

In [0]:
scaler = preprocessing.StandardScaler().fit(X_smo_train)
X_smo_train_scaled = scaler.transform(X_smo_train)
X_test_scaled = scaler.transform(X_test)

ii. Modeling

In [0]:
logisticReg4 = LogisticRegression(max_iter=20000)
logisticReg4.fit(X_smo_train_scaled, y_smo_train)

iii. Prediction

In [0]:
logistic4_pred = logisticReg4.predict(X_test_scaled)
logistic4_pred_probs = logisticReg4.predict_proba(X_test_scaled)

iv. Evaluation

In [0]:
eval_class(y_test, logistic4_pred)
plot_roc(y_test, logistic4_pred_probs, 'Linear Regression')
confision_matrix(y_test, logistic4_pred, 'Linear Regression')

### (3) Random Forest

i. Modeling

In [0]:
randomforest4 = RandomForestClassifier(n_estimators=100).fit(X_smo_train_scaled, y_smo_train)

ii. Prediction

In [0]:
randomforest4_pred = randomforest4.predict(X_test_scaled)
randomforest4_pred_probs = randomforest4.predict_proba(X_test_scaled)

iii. Evaluation

In [0]:
eval_class(y_test, randomforest4_pred)
plot_roc(y_test, randomforest4_pred_probs, 'Random Forest')
confision_matrix(y_test, randomforest4_pred, 'Random Forest')

### (4) CatBoost

i. Modeling

In [0]:
# Model params
params_4 = {
    'loss_function': 'MultiClass',
    'eval_metric': 'Accuracy',
    'verbose': 10,
    'random_seed': 24,
    # 'depth':6, 
    'learning_rate':0.0001,
    'use_best_model': True,
    'l2_leaf_reg': 10,
    'bagging_temperature': 3,
    'od_type': "Iter",
    'od_wait': 100
}

catboostCls4 = catboost.CatBoostClassifier(**params_4)
catboostCls4.fit(X_smo_train_scaled, y_smo_train,
             eval_set = (X_test_scaled, y_test),
             use_best_model = True,
             plot=True)

ii. Prediction

In [0]:
cat4_pred = catboostCls4.predict(X_test_scaled)
cat4_prob = catboostCls4.predict_proba(X_test_scaled)

iii. Evaluation

In [0]:
eval_class(y_test, cat4_pred)
plot_roc(y_test, cat4_prob, 'CatBoost')
confision_matrix(y_test, cat4_pred, 'CatBoost')

## 9.Conclusion

1. Under Sample Majority + Logistic Regression gives best specificity. Which is 0.5806451612903226 (only fitbit data gives 0.5657894736842105).
2. Best AUC given by SMOTE + Logistic Regression, with auc = 0.7.
3. Under sample majority in general helps model detect unhappy case.